In [0]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
import logging
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('fmcg').getOrCreate()

In [0]:
%run /Workspace/apache-spark/databricks-project-fmcg-sports/utils/utilities

In [0]:
# read customer data
file_path = f'{s3_bucket}/{data_source}/customers.csv'
df_cust_bronze = (
    spark.read
    .format('csv')
    .option('header','true')
    .option('inferSchema','true')
    .load(file_path)
)

df_cust_bronze.display()


### As I see following issue with imported data
0. duplicate records
1. customer id is intiger it should be string, 
2. customer_name is not consistent it has leeding spaces, capatilisaiton etc
3. city name has speling mistakes.


In [0]:
# Duplicates
# check if there any duplicate
df_cust_bronze.groupBy('customer_id').count().filter(f.col('count') > 1).display()

# remove these duplicates
df_cust_silver = df_cust_bronze.dropDuplicates(['customer_id'])

In [0]:
# Create a city mapping df to join with original df
df_city = spark.createDataFrame(clean_city, ["city", "clean_city"])

# join both table and get city and trim the first name
df_cust_silver = (
    df_customers_bronze
    .join(df_city, on="city", how='left')
    .select("customer_id", "customer_name", f.col('clean_city').alias('city'))
).withColumns({
    'customer_name': f.initcap(f.trim(f.col('customer_name')))
})
df_cust_silver.select('city').distinct().show()

In [0]:
df_cust_silver.filter(f.col("city").isNull()).show()